<a href="https://colab.research.google.com/github/gowripreetham/SJSU_Data_Mining_Unsloth-Assignment/blob/main/colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 1: Full Fine-tuning with SmolLM2-135M using Unsloth

### Overview
This notebook demonstrates **full parameter fine-tuning** using Unsloth with the SmolLM2-135M model.

#### What is Full Fine-tuning?
- Updates **ALL** model parameters during training (unlike LoRA which only updates adapters)
- Provides maximum performance but requires more memory and time
- Best for when you need the highest quality results

#### Key Features:
- Model: `unsloth/SmolLM2-135M-Instruct` (135 million parameters)
- Dataset: **tatsu-lab/alpaca** (52K examples)
- Training time: ~2–3 minutes on free Colab T4 GPU
- Task: General instruction following

#### What You’ll Learn:
1. How to load and configure a model for full fine-tuning
2. Dataset preparation and formatting
3. Training configuration and execution
4. Inference and model evaluation
5. Saving and exporting the model


In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

### Step 2: Verify GPU and Setup

In [ ]:
import torch
from unsloth import FastLanguageModel

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
CUDA version: 12.6


### Step 3: Load Model for Full Fine-tuning

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-135M-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print('Model loaded successfully!')

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model loaded successfully!


### Step 4: Prepare Model for Training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    use_rslora=False,
    r=16,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)
print('Model prepared for training!')

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.11.2 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Model prepared for training!


### Step 5: Load and Prepare Dataset (Tatsu-Lab Alpaca)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('tatsu-lab/alpaca', split='train[:200]')
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


### Step 6: Format Dataset for Training

In [ ]:
EOS_TOKEN = tokenizer.eos_token
def format_example(example):
    return {
        'text': f"Below is an instruction that describes a task, paired with an input that provides further context.\n\n### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']} {EOS_TOKEN}"
    }
dataset = dataset.map(format_example)
print(dataset[0]['text'][:500])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction:
Give three tips for staying healthy.

### Input:


### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. <|im_end|>


In [ ]:
# Tokenize dataset correctly for causal LM fine-tuning
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_seq_length,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

print("✅ Tokenization complete! Example keys:", tokenized_dataset.column_names)
print("Example tokenized entry:\n", tokenized_dataset[0])


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Tokenization complete! Example keys: ['input_ids', 'attention_mask', 'labels']
Example tokenized entry:
 {'input_ids': [19798, 314, 354, 5785, 338, 6601, 253, 3856, 28, 20054, 351, 354, 3007, 338, 2433, 2030, 2468, 30, 198, 198, 3757, 20880, 42, 198, 26533, 1296, 5608, 327, 9286, 2458, 30, 198, 198, 3757, 18100, 42, 1116, 198, 3757, 14212, 42, 198, 33, 30, 36693, 253, 8609, 2714, 284, 919, 2090, 288, 1453, 7568, 282, 5574, 284, 5136, 30, 3717, 34, 30, 15382, 5578, 288, 1446, 469, 1248, 3212, 284, 1837, 30, 3717, 35, 30, 5399, 2001, 2801, 284, 2125, 253, 5707, 2801, 8034, 30, 216, 2, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49152, 49

### Step 7: Configure Training Parameters

In [ ]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        logging_steps=10,
        output_dir='outputs',
    ),
)
print('Trainer configured successfully!')

The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer configured successfully!


### Step 8: Train the Model

In [ ]:
trainer_stats = trainer.train()
print('Training complete!')

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 5 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 921,600 of 135,437,184 (0.68% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,540.165400
20,435.779200
30,274.516200
40,141.592700
50,47.776600
60,19.770400


Training complete!


### Step 9: Test the Model (Inference)

In [ ]:
FastLanguageModel.for_inference(model)
prompts = [
    'Write a haiku about AI.',
    'Explain what reinforcement learning is.',
    'What are the benefits of eating vegetables?',
]
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(**inputs, max_new_tokens=100)
    print('\nPrompt:', prompt)
    print('Response:', tokenizer.decode(outputs[0], skip_special_tokens=True))


Prompt: Write a haiku about AI.
Response: Write a haiku about AI.

Prompt: Explain what reinforcement learning is.
Response: Explain what reinforcement learning is.

In reinforcement learning, the goal is to create a model that can learn from experience and make decisions based on the data it has been trained on. This can be achieved through various techniques, such as:

1. **Epistemic learning**: This involves learning from the current state of the system, rather than trying to predict the future. This can be achieved through techniques like backpropagation, stochastic gradient descent, or stochastic gradient clipping.
2. **Epistemic learning with reinforcement

Prompt: What are the benefits of eating vegetables?
Response: What are the benefits of eating vegetables?

A: Eating vegetables can help reduce the risk of chronic diseases such as heart disease, type 2 diabetes, and certain types of cancer. They are also rich in vitamins, minerals, and fiber, which can help keep your body he

### Step 10: Save the Model

In [ ]:
model.save_pretrained('smollm2_finetuned_tatsulab')
tokenizer.save_pretrained('smollm2_finetuned_tatsulab')
print('Model saved successfully!')

Model saved successfully!
